# Airbnb San Francisco Data Cleaning

This notebook loads and inspects Airbnb's San Francisco raw "listings" dataset.  

Datasets included:
- listings.csv

Goal: identify issues, clean data, and prepare it for EDA.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display all columns when displaying data
pd.set_option('display.max_columns', None)

In [ ]:
listings = pd.read_csv('/Users/mohammedzareef-mustafa/Downloads/Tech Career/Tech Projects/Projects/airbnb-sf-eda/data/raw/listings.csv')

In [ ]:
listings.head()

In [ ]:
listings.info()

In [ ]:
listings.isna().sum().sort_values(ascending=False).head(20)

In [ ]:
date_cols = ['last_scraped', 'host_since', 'first_review', 'last_review']
listings[date_cols]

In [ ]:
# convert columns that should be dates
date_cols = ['last_scraped', 'host_since', 'first_review', 'last_review']

for col in date_cols:
    listings[col] = pd.to_datetime(listings[col], errors='coerce')

# quick check
listings[date_cols]

In [ ]:
money_cols = ['price', 'estimated_revenue_l365d']
listings[money_cols]

In [ ]:
# clean up money fields (strip $, commas, turn into float)
money_cols = ['price', 'estimated_revenue_l365d']

for col in money_cols:
    listings[col] = (
        listings[col]
        .astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .replace('nan', np.nan)
        .astype(float)
    )

# quick check
listings[money_cols]

In [ ]:
rate_cols = ['host_response_rate', 'host_acceptance_rate']
listings[rate_cols]

In [ ]:
# turn percentage style strings into numbers
rate_cols = ['host_response_rate', 'host_acceptance_rate']

for col in rate_cols:
    listings[col] = (
        listings[col]
        .str.rstrip('%')
        .astype(float)
    )

# quick check
listings[rate_cols]

In [ ]:
bool_cols = [
    'host_is_superhost',
    'host_has_profile_pic',
    'host_identity_verified',
    'has_availability',
    'instant_bookable'
]
listings[bool_cols]

In [ ]:
# clean boolean columns safely
bool_cols = [
    'host_is_superhost',
    'host_has_profile_pic',
    'host_identity_verified',
    'has_availability',
    'instant_bookable'
]

for col in bool_cols:
    listings[col] = (
        listings[col]
        .astype(str)      # ensure string
        .str.lower()      # normalize
        .str.strip()      # remove odd whitespace
        .replace({'nan': None})  # handle missing
        .map({'t': True, 'f': False})
    )

# check results
listings[bool_cols]

In [ ]:
# high level check
listings.info()

In [ ]:
# quick look at top 20 columns with missing values
listings.isna().sum().sort_values(ascending=False).head(20)

In [ ]:
# drop columns that are fully empty
cols_to_drop = [
    'neighbourhood_group_cleansed',
    'calendar_updated'
]

listings = listings.drop(columns=cols_to_drop)

# checking if empty columns are gone
listings.columns


In [ ]:
# saving a clean working copy
listings_clean = listings.copy()

# keep rows with a valid price
listings_clean = listings_clean[listings_clean['price'].notna()]

# keep rows that have at least one review or a rating
listings_clean = listings_clean[
    (listings_clean['number_of_reviews'] > 0) |
    (listings_clean['review_scores_rating'].notna())
]

# check how many rows we have now
listings.shape[0], listings_clean.shape[0]

In [ ]:
# checking if these empty rows are filtered out
listings_clean[['price', 'number_of_reviews', 'review_scores_rating']].isna().sum()

In [ ]:
# quick look at top 20 columns with missing values in the clean dataset
listings_clean.isna().sum().sort_values(ascending=False).head(20)

In [ ]:
# fill key capacity fields using median by room type
for col in ['bedrooms', 'beds', 'bathrooms']:
    listings_clean[col] = listings_clean.groupby('room_type')[col]\
                                        .transform(lambda x: x.fillna(x.median()))

# quick check
listings_clean[['room_type', 'bedrooms', 'beds', 'bathrooms']].isna().sum()

In [ ]:
# Adding valuable columns using basic formulas

listings_clean['price_per_person'] = listings_clean['price'] / listings_clean['accommodates']

# days since host joined
listings_clean['host_tenure_days'] = (
    listings_clean['last_scraped'] - listings_clean['host_since']
).dt.days

# simple rating percent (0–100), if rating exists
listings_clean['rating_pct'] = (listings_clean['review_scores_rating'] / 5) * 100

# check a sample
listings_clean[['price', 'accommodates', 'price_per_person',
                'host_since', 'host_tenure_days',
                'review_scores_rating', 'rating_pct']].head()

In [ ]:
# rough count of amenities from the text field
def count_amenities(s):
    if pd.isna(s):
        return 0
    inner = s.strip('[]')
    if inner == '':
        return 0
    return len(inner.split(','))

listings_clean['amenities_count'] = listings_clean['amenities'].apply(count_amenities)

# quick check
listings_clean[['amenities', 'amenities_count']].head()

In [ ]:
# drop url/image fields we do not need for analysis
drop_cols = [
    'listing_url', 'host_url', 'picture_url',
    'host_thumbnail_url', 'host_picture_url'
]

listings_clean = listings_clean.drop(columns=drop_cols)

# sanity check
listings_clean.columns

In [ ]:
# save cleaned listings dataset to use in the EDA notebook
listings_clean.to_csv('/Users/mohammedzareef-mustafa/Downloads/Tech Career/Tech Projects/Projects/airbnb-sf-eda/data/clean/listings_clean.csv', index=False)

listings_clean.head()